<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/chmntz_Lab1/Lab1/chmntz_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Matrix factorization**
Carl **Chemnitz**

# **Abstract**


# **Set up environment**
To have access to the neccessary modules you have to run this cell.

In [310]:
from google.colab import files

import numpy as np
import scipy.sparse
import math

# **Introduction**

## **Mandatory assignment**
1. Function: **sparse matrix-vector product**
**Input**: vector $x$, sparse (real, quadratic) matrix $A$: CRS arrays val, col_idx, row_ptr.\
**Output**: matrix-vector product $b=Ax$.\
**Test**: verify accuracy against dense matrix-vector product.

2. Function: QR factorization
**Input**: (real, quadratic, invertible) matrix $A$.\
**Output**: orthogonal matrix $Q$, upper traingular matrix $R$, such that $A=QR$.\
**Test**: $R$ upper traingular, Forbenius norms $||Q^TQ-I||_F$, $||QR-A||_F$.

3. Function: direct solver $Ax=b$
**Input**: (real, quadratic) matrix $A$, vector $b$.\
**Output**: vector $x=A^{-1}b$.\
**Test**: residual $||Ax-b||$, and $||x-y||$ where $y$ is a manufactured solution $b=Ay$.\

# **Method**
The implementation of each assignment.

## Sparse matrix-vector product
A sparse matrix is a matrix were most of its elements are zero. To optimize memory and efficiency of matrix-vector multiplication, *compressed row storage* data structures can be used. It consists of three arrays.
* `val`, the non-zero values.
* `col_idx`, their respective column indices.
* `row_ptr`, pointers to the start of each row.

`sparse_matrix_vector_product` was implemented based on algorithm **5.9** from Chapter 5 of Methods in Computational Science, and adjusted to 0-indexing.

In [311]:
def sparse_matrix_vector_product (val, col_idx, row_ptr, x) -> np.array:
    b = np.zeros(len(row_ptr)-1)
    for i in range(len(row_ptr)-1):
        for j in range(row_ptr[i], row_ptr[i+1]):
            b[i] += val[j] * x[col_idx[j]]
    return b


The standard (dense) matrix-vector multiplication algorithm was implemented based on algorithm **3.2** from Chapter 3 of Methods in Computational Science.

In [312]:
def matrix_vector_product (A, x) -> np.array:
    b = np.zeros(len(x))
    for i in range(0, len(A[:,0])):
        for j in range(0, len(x)):
            b[i] += A[i,j] * x[j]
    return b

Random sparse matrices and vectors were constructed and multiplied using `sparse_matrix_vector_product` and `matrix_vector_product`, and then compared.

In [313]:
for i in range(5):
    A = scipy.sparse.random(i+5, i+5, density=0.3, format='csr')
    x = np.random.rand(i+5)

    sparse_b = sparse_matrix_vector_product(A.data, A.indices, A.indptr, x)
    dense_b = matrix_vector_product(A.todense(), x)
    print(f"Test {i},   Verification {'success' if (dense_b == sparse_b).all() else 'failure'}")


Test 0,   Verification success
Test 1,   Verification success
Test 2,   Verification success
Test 3,   Verification success
Test 4,   Verification success


## QR factorization